# 🚀 SnaX IA v3 - Treinamento no Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seu-usuario/snax-ia-v3/blob/main/SnaX_IA_v3_Colab.ipynb)

Este notebook permite treinar o SnaX IA v3 diretamente no Google Colab.

## 1️⃣ Setup

Instalando dependências e clonando o repositório:

In [ ]:
!pip install torch numpy tqdm datasets sentencepiece transformers
!git clone https://github.com/seu-usuario/snax-ia-v3
%cd snax-ia-v3

## 2️⃣ Verificação do Ambiente

Verificando se temos GPU disponível:

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

## 3️⃣ Configuração do Modelo

Definindo hiperparâmetros:

In [ ]:
config = {
    "vocab_size": 8192,
    "hidden_size": 384,
    "num_layers": 6,
    "num_heads": 6,
    "num_kv_heads": 2,
    "max_seq_len": 512,
    "batch_size": 32,
    "grad_accum_steps": 4,
    "learning_rate": 3e-4,
    "warmup_steps": 1000,
    "max_steps": 20000,
    "eval_interval": 500,
    "save_interval": 1000,
    "mixed_precision": True
}

## 4️⃣ Treinamento

Iniciando o treinamento do modelo:

In [ ]:
from model_v3 import SnaXIA_v3
from tokenizer_v3 import SnaXTokenizer_v3
from train_v3 import Trainer
from datasets import load_dataset
from torch.utils.data import DataLoader

# Criar tokenizer
tokenizer = SnaXTokenizer_v3()

# Carregar dataset
dataset = load_dataset("wikipedia", "20220301.pt", split="train")

# Criar modelo
model = SnaXIA_v3(config)

# Criar trainer
trainer = Trainer(model, tokenizer, config)

# Treinar
trainer.train(train_loader)

## 5️⃣ Testes

Testando o modelo treinado:

In [ ]:
from chat_v3 import SnaXChat

# Carregar modelo
chat = SnaXChat("checkpoints/best_checkpoint.pt")

# Testar geração
prompts = [
    "Python é uma linguagem",
    "A capital do Brasil é",
    "Machine learning é"
]

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    response = chat.generate(prompt)
    print(f"Resposta: {response}")

## 6️⃣ Exportação para Mobile

Exportando o modelo para ONNX com quantização INT8:

In [ ]:
from export_mobile import convert_to_onnx, quantize_onnx, benchmark_model

# Exportar para ONNX
convert_to_onnx("checkpoints/best_checkpoint.pt", "snaxia_v3.onnx")

# Quantizar para INT8
quantize_onnx("snaxia_v3.onnx", "snaxia_v3_int8.onnx")

# Benchmark
print("\nBenchmark FP32:")
benchmark_model("snaxia_v3.onnx")

print("\nBenchmark INT8:")
benchmark_model("snaxia_v3_int8.onnx")

## 7️⃣ Download

Baixando os arquivos gerados:

In [ ]:
from google.colab import files

# Comprimir arquivos
!zip -r snaxia_v3_models.zip checkpoints/*.pt *.onnx tokenizer_v3.model

# Download
files.download('snaxia_v3_models.zip')